In [1]:
import numpy as np
import pandas as pd
from glob import glob
import shutil, os
import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedGroupKFold
from tqdm.notebook import tqdm
import seaborn as sns

In [2]:
train_df = pd.read_csv(r'E:\XR-AI\datasets\vinbigdata\train.csv')
train_df['image_path'] = r'E:\XR-AI\datasets\vinbigdata\train' + train_df.image_id+'.png'
#train_df = train_df[train_df.class_id!=14].reset_index(drop = True)
train_df.head(100)

,image_id,class_name,class_id,rad_id,x_min,y_min,x_max,y_max,width,height,image_path
0,50a418190bc3fb1ef1633bf9678929b3,No finding,14,R11,NaN,NaN,NaN,NaN,2332,2580,/notebooks/file/vinbigdata/train/50a418190bc3f...
1,21a10246a5ec7af151081d0cd6d65dc9,No finding,14,R7,NaN,NaN,NaN,NaN,2954,3159,/notebooks/file/vinbigdata/train/21a10246a5ec7...
2,9a5094b2563a1ef3ff50dc5c7ff71345,Cardiomegaly,3,R10,691.0,1375.0,1653.0,1831.0,2080,2336,/notebooks/file/vinbigdata/train/9a5094b2563a1...
3,051132a778e61a86eb147c7c6f564dfe,Aortic enlargement,0,R10,1264.0,743.0,1611.0,1019.0,2304,2880,/notebooks/file/vinbigdata/train/051132a778e61...
4,063319de25ce7edb9b1c6b8881290140,No finding,14,R10,NaN,NaN,NaN,NaN,2540,3072,/notebooks/file/vinbigdata/train/063319de25ce7...
...,...,...,...,...,...,...,...,...,...,...,...
95,37233691ba64ed88b4e05882d7e41d61,Pulmonary fibrosis,13,R10,2154.0,855.0,2691.0,1272.0,3320,3408,/notebooks/file/vinbigdata/train/37233691ba64e...
96,69f5d29210d98df18120534adb990291,No finding,14,R15,NaN,NaN,NaN,NaN,3000,3000,/notebooks/file/vinbigdata/train/69f5d29210d98...
97,189e409214f5a4639d821a34064d9140,No finding,14,R6,NaN,NaN,NaN,NaN,1994,2430,/notebooks/file/vinbigdata/train/189e409214f5a...
98,c5c36ce7f8adbd045bfe59a325eeae47,Aortic enlargement,0,R8,1333.0,757.0,1606.0,1118.0,2608,2384,/notebooks/file/vinbigdata/train/c5c36ce7f8adb...


In [3]:
train_df['x_min'] = train_df.apply(lambda row: (row.x_min)/row.width, axis =1)
train_df['y_min'] = train_df.apply(lambda row: (row.y_min)/row.height, axis =1)

train_df['x_max'] = train_df.apply(lambda row: (row.x_max)/row.width, axis =1)
train_df['y_max'] = train_df.apply(lambda row: (row.y_max)/row.height, axis =1)

train_df['x_mid'] = train_df.apply(lambda row: (row.x_max+row.x_min)/2, axis =1)
train_df['y_mid'] = train_df.apply(lambda row: (row.y_max+row.y_min)/2, axis =1)

train_df['w'] = train_df.apply(lambda row: (row.x_max-row.x_min), axis =1)
train_df['h'] = train_df.apply(lambda row: (row.y_max-row.y_min), axis =1)

train_df['area'] = train_df['w']*train_df['h']
train_df.head()

,image_id,class_name,class_id,rad_id,x_min,y_min,x_max,y_max,width,height,image_path,x_mid,y_mid,w,h,area
0,50a418190bc3fb1ef1633bf9678929b3,No finding,14,R11,NaN,NaN,NaN,NaN,2332,2580,/notebooks/file/vinbigdata/train/50a418190bc3f...,NaN,NaN,NaN,NaN,NaN
1,21a10246a5ec7af151081d0cd6d65dc9,No finding,14,R7,NaN,NaN,NaN,NaN,2954,3159,/notebooks/file/vinbigdata/train/21a10246a5ec7...,NaN,NaN,NaN,NaN,NaN
2,9a5094b2563a1ef3ff50dc5c7ff71345,Cardiomegaly,3,R10,0.332212,0.588613,0.794712,0.783818,2080,2336,/notebooks/file/vinbigdata/train/9a5094b2563a1...,0.563462,0.686216,0.462500,0.195205,0.090283
3,051132a778e61a86eb147c7c6f564dfe,Aortic enlargement,0,R10,0.548611,0.257986,0.699219,0.353819,2304,2880,/notebooks/file/vinbigdata/train/051132a778e61...,0.623915,0.305903,0.150608,0.095833,0.014433
4,063319de25ce7edb9b1c6b8881290140,No finding,14,R10,NaN,NaN,NaN,NaN,2540,3072,/notebooks/file/vinbigdata/train/063319de25ce7...,NaN,NaN,NaN,NaN,NaN


In [4]:
features = ['x_min', 'y_min', 'x_max', 'y_max', 'x_mid', 'y_mid', 'w', 'h', 'area']
X = train_df[features]
y = train_df['class_id']
X.shape, y.shape

((67914, 9), (67914,))

In [5]:
class_ids, class_names = list(zip(*set(zip(train_df.class_id, train_df.class_name))))
classes = list(np.array(class_names)[np.argsort(class_ids)])
classes = list(map(lambda x: str(x), classes))
classes
train_df.head()

,image_id,class_name,class_id,rad_id,x_min,y_min,x_max,y_max,width,height,image_path,x_mid,y_mid,w,h,area
0,50a418190bc3fb1ef1633bf9678929b3,No finding,14,R11,NaN,NaN,NaN,NaN,2332,2580,/notebooks/file/vinbigdata/train/50a418190bc3f...,NaN,NaN,NaN,NaN,NaN
1,21a10246a5ec7af151081d0cd6d65dc9,No finding,14,R7,NaN,NaN,NaN,NaN,2954,3159,/notebooks/file/vinbigdata/train/21a10246a5ec7...,NaN,NaN,NaN,NaN,NaN
2,9a5094b2563a1ef3ff50dc5c7ff71345,Cardiomegaly,3,R10,0.332212,0.588613,0.794712,0.783818,2080,2336,/notebooks/file/vinbigdata/train/9a5094b2563a1...,0.563462,0.686216,0.462500,0.195205,0.090283
3,051132a778e61a86eb147c7c6f564dfe,Aortic enlargement,0,R10,0.548611,0.257986,0.699219,0.353819,2304,2880,/notebooks/file/vinbigdata/train/051132a778e61...,0.623915,0.305903,0.150608,0.095833,0.014433
4,063319de25ce7edb9b1c6b8881290140,No finding,14,R10,NaN,NaN,NaN,NaN,2540,3072,/notebooks/file/vinbigdata/train/063319de25ce7...,NaN,NaN,NaN,NaN,NaN


In [6]:
sgkf = StratifiedGroupKFold(n_splits=10, shuffle=True, random_state=42)
train_df['fold'] = -1
for fold, (train_idx, val_idx) in enumerate(sgkf.split(train_df,train_df['class_name'], groups = train_df.image_id.tolist())):
    train_df.loc[val_idx, 'fold'] = fold

REMEMBER TO CHANGE FOLD VALUE

In [7]:
fold = 5

train_files = []
val_files   = []
val_files += list(train_df[train_df.fold==fold].image_path.unique())
train_files += list(train_df[train_df.fold!=fold].image_path.unique())
len(train_files), len(val_files)

(13495, 1505)

In [ ]:
os.makedirs('/notebooks/vinbigdata_skfold_6/labels/train', exist_ok = True)
os.makedirs('/notebooks/vinbigdata_skfold_6/labels/val', exist_ok = True)
os.makedirs('/notebooks/vinbigdata_skfold_6/images/train', exist_ok = True)
os.makedirs('/notebooks/vinbigdata_skfold_6/images/val', exist_ok = True)
#label_dir = r'E:\XR-AI\datasets\vinbigdata\label\labels'
WBF_label_dir = '/notebooks/file/vinbigdata/WBF_labels'
for file in tqdm(train_files):
    shutil.copy(file, '/notebooks/vinbigdata_skfold_6/images/train')
    filename = file.split('/')[-1].split('.')[0]
    shutil.copy(os.path.join(WBF_label_dir, filename+'.txt'), '/notebooks/vinbigdata_skfold_6/labels/train')

for file in tqdm(val_files):
    shutil.copy(file, '/notebooks/vinbigdata_skfold_6/images/val')
    filename = file.split('/')[-1].split('.')[0]
    shutil.copy(os.path.join(WBF_label_dir, filename+'.txt'), '/notebooks/vinbigdata_skfold_6/labels/val')
class_ids, class_names = list(zip(*set(zip(train_df.class_id, train_df.class_name))))
classes = list(np.array(class_names)[np.argsort(class_ids)])
classes = list(map(lambda x: str(x), classes))
classes

  0%|          | 0/13495 [00:00<?, ?it/s]

In [ ]:
from os import listdir

from os.path import isfile, join
import yaml

os.makedirs("/notebooks/vinbigdata_skfold_6",exist_ok= True)
cwd = '/notebooks/vinbigdata_skfold_6'


with open(join( cwd , 'train.txt'), 'w') as f:
    for path in glob('/notebooks/vinbigdata_skfold_6/images/train/*'):
        f.write(path+'\n')

with open(join( cwd , 'val.txt'), 'w') as f:
    for path in glob('/notebooks/vinbigdata_skfold_6/images/val/*'):
        f.write(path+'\n')

data = dict(
    train =  join( cwd , 'train.txt') ,
    val   =  join( cwd , 'val.txt' ),
    nc    = 15,
    names = classes
    )

with open(join( cwd , 'vinbigdata_skfold6.yaml'), 'w') as outfile:
    yaml.dump(data, outfile, default_flow_style=False)

f = open(join( cwd , 'vinbigdata_skfold6.yaml'), 'r')
print('\nyaml:')
print(f.read())

In [1]:
#Install yolov8
!pip install ultralytics

#Install & setup ClearML env
!pip install clearml
%env CLEARML_WEB_HOST=https://app.clear.ml
%env CLEARML_API_HOST=https://api.clear.ml
%env CLEARML_FILES_HOST=https://files.clear.ml
%env CLEARML_API_ACCESS_KEY=1E8D8P0FEKHX4V7LDQ4X
%env CLEARML_API_SECRET_KEY=2NXirhWNHuaHWsKkRD43onIyE8OvJH2IvPTEavCnQmeWlcPecn

#Install optuna 
!pip install optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 511.0/511.0 kB 12.9 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 11.0 MB/s eta 0:00:00 0:00:01
env: CLEARML_WEB_HOST=https://app.clear.ml
env: CLEARML_API_HOST=https://api.clear.ml
env: CLEARML_FILES_HOST=https://files.clear.ml
env: CLEARML_API_ACCESS_KEY=1E8D8P0FEKHX4V7LDQ4X
env: CLEARML_API_SECRET_KEY=2NXirhWNHuaHWsKkRD43onIyE8OvJH2IvPTEavCnQmeWlcPecn
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.7/365.7 kB 41.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.3/212.3 kB 56.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 27.6 MB/s eta 0:00:00


In [2]:
from clearml import Task

task = Task.init(
    project_name='M-AI',    # project name of at least 3 characters
    task_name='kfold_6', # task name of at least 3 characters
    task_type='training',
    tags="AdamW",
    reuse_last_task_id=True,
    continue_last_task=True,
    auto_connect_arg_parser=True,
    auto_connect_frameworks=True,
    auto_resource_monitoring=True,
    auto_connect_streams=True,
)

ClearML Task: created new task id=5963e511de004d2bba9189e7ecbccc0c
ClearML results page: https://app.clear.ml/projects/1fc80985a47343f2ab21cc32e73a02e1/experiments/5963e511de004d2bba9189e7ecbccc0c/output/log
2023-04-10 01:38:43,691 - clearml.Task - INFO - Storing jupyter notebook directly as code
ClearML Monitor: Could not detect iteration reporting, falling back to iterations as seconds-from-start


In [3]:
from ultralytics import YOLO
model = YOLO("/notebooks/yolov8l.yaml")
model.train(data = "/notebooks/vinbigdata_skfold_6/vinbigdata_skfold6.yaml"
            ,project = 'M-AI'
            ,name = 'kfold_6'
            ,epochs = 300
            ,patience = 50
            ,optimizer = 'AdamW'
            ,momentum = 0.9
            ,verbose = True
            ,batch = 32
            ,lr0 = 0.001
            ,imgsz = 1024
            ,seed = 42
            )
task.close()


                   from  n    params  module                                       arguments                     
  0                  -1  1      1856  ultralytics.nn.modules.Conv                  [3, 64, 3, 2]                 
  1                  -1  1     73984  ultralytics.nn.modules.Conv                  [64, 128, 3, 2]               
  2                  -1  3    279808  ultralytics.nn.modules.C2f                   [128, 128, 3, True]           
  3                  -1  1    295424  ultralytics.nn.modules.Conv                  [128, 256, 3, 2]              
  4                  -1  6   2101248  ultralytics.nn.modules.C2f                   [256, 256, 6, True]           
  5                  -1  1   1180672  ultralytics.nn.modules.Conv                  [256, 512, 3, 2]              
  6                  -1  6   8396800  ultralytics.nn.modules.C2f                   [512, 512, 6, True]           
  7                  -1  1   2360320  ultralytics.nn.modules.Conv                  [512

  0%|          | 0.00/755k [00:00<?, ?B/s]

Overriding model.yaml nc=80 with nc=15

                   from  n    params  module                                       arguments                     
  0                  -1  1      1856  ultralytics.nn.modules.Conv                  [3, 64, 3, 2]                 
  1                  -1  1     73984  ultralytics.nn.modules.Conv                  [64, 128, 3, 2]               
  2                  -1  3    279808  ultralytics.nn.modules.C2f                   [128, 128, 3, True]           
  3                  -1  1    295424  ultralytics.nn.modules.Conv                  [128, 256, 3, 2]              
  4                  -1  6   2101248  ultralytics.nn.modules.C2f                   [256, 256, 6, True]           
  5                  -1  1   1180672  ultralytics.nn.modules.Conv                  [256, 512, 3, 2]              
  6                  -1  6   8396800  ultralytics.nn.modules.C2f                   [512, 512, 6, True]           
  7                  -1  1   2360320  ultralytic

FileNotFoundError: [34m[1mval: [0mError loading data from /notebooks/vinbigdata_skfold_6/val.txt
See https://github.com/ultralytics/yolov5/wiki/Train-Custom-Data

In [3]:
from ultralytics import YOLO

model = YOLO("/notebooks/M-AI/kfold_6/weights/last.pt")
model.train(data = "/notebooks/vinbigdata_skfold_6/vinbigdata_skfold6.yaml"
            ,project = 'M-AI'
            ,name = 'kfold_6'
            ,resume = True)
task.close()


2023-04-09 05:53:31,120 - clearml.model - INFO - Selected model id: c4cc8492c75e498ab5616bdc910d4c37


Ultralytics YOLOv8.0.71 🚀 Python-3.9.16 torch-1.12.1+cu116 CUDA:0 (NVIDIA RTX A6000, 48685MiB)
yolo/engine/trainer: task=detect, mode=train, model=/notebooks/M-AI/kfold_5/weights/last.pt, data=/notebooks/vinbigdata_skfold_5/vinbigdata_skfold5.yaml, epochs=300, patience=50, batch=32, imgsz=1024, save=True, save_period=-1, cache=False, device=None, workers=8, project=M-AI, name=kfold_5, exist_ok=False, pretrained=False, optimizer=AdamW, verbose=True, seed=42, deterministic=True, single_cls=False, image_weights=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, line_thickness=3, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, boxes=True, fo

  0%|          | 0.00/755k [00:00<?, ?B/s]

TensorBoard: Start with 'tensorboard --logdir M-AI/kfold_5', view at http://localhost:6006/

                   from  n    params  module                                       arguments                     
  0                  -1  1      1856  ultralytics.nn.modules.Conv                  [3, 64, 3, 2]                 
  1                  -1  1     73984  ultralytics.nn.modules.Conv                  [64, 128, 3, 2]               
  2                  -1  3    279808  ultralytics.nn.modules.C2f                   [128, 128, 3, True]           
  3                  -1  1    295424  ultralytics.nn.modules.Conv                  [128, 256, 3, 2]              
  4                  -1  6   2101248  ultralytics.nn.modules.C2f                   [256, 256, 6, True]           
  5                  -1  1   1180672  ultralytics.nn.modules.Conv                  [256, 512, 3, 2]              
  6                  -1  6   8396800  ultralytics.nn.modules.C2f                   [512, 512, 6, True]       

Resuming training from /notebooks/M-AI/kfold_5/weights/last.pt from epoch 263 to 300 total epochs
Image sizes 1024 train, 1024 val
Using 8 dataloader workers
Logging results to M-AI/kfold_5
Starting training for 300 epochs...
Exception ignored in: <function _after_at_fork_child_reinit_locks at 0x7f95aa5e1ee0>Exception ignored in: 
<function _after_at_fork_child_reinit_locks at 0x7f95aa5e1ee0>
Traceback (most recent call last):
  File "/usr/lib/python3.9/logging/__init__.py", line 255, in _after_at_fork_child_reinit_locks
Exception ignored in: Traceback (most recent call last):
    Exception ignored in:   File "/usr/lib/python3.9/logging/__init__.py", line 255, in _after_at_fork_child_reinit_locks
<function _after_at_fork_child_reinit_locks at 0x7f95aa5e1ee0>handler._at_fork_reinit()<function _after_at_fork_child_reinit_locks at 0x7f95aa5e1ee0>
    
Traceback (most recent call last):
  File "/usr/lib/python3.9/logging/__init__.py", line 894, in _at_fork_reinit
  File "/usr/lib/python3.9